In [1]:
# IEEE Challenge

In [2]:
import pandas as pd
from src import paths
from src.read_data import DataReader
from src.data_preprocessor import DataPreprocessor

In [3]:
labelled = DataReader.get_posts_with_labels()

In [4]:
labelled.head()

,post,post_risk
0,Just kill me. Please! Just end my life! I beg ...,ideation
1,"There is no today, nor tommorrow. I dont have ...",behavior
2,Life so full of contradictions that it's not w...,indicator
3,I think I'm going to kill myself soon. I don't...,behavior
4,whats the point of living. no really. is there...,ideation


In [5]:
labelled['post_risk'].value_counts()

post_risk
ideation     190
behavior     140
indicator    129
attempt       41
Name: count, dtype: int64

In [7]:
labelled = DataPreprocessor.preprocess(labelled)
labelled.head()

,post,post_risk,word_count
0,Just kill me. Please! Just end my life! I beg ...,ideation,14
1,"There is no today, nor tommorrow. I dont have ...",behavior,282
2,Life so full of contradictions that it's not w...,indicator,311
3,I think I'm going to kill myself soon. I don't...,behavior,54
4,whats the point of living. no really. is there...,ideation,142


In [9]:
labelled.groupby('post_risk')['word_count'].describe()

,count,mean,std,min,25%,50%,75%,max
post_risk,,,,,,,,
attempt,41.0,338.439024,426.966805,11.0,55.0,179.0,447.00,2095.0
behavior,140.0,177.321429,186.431242,2.0,55.0,94.0,230.25,830.0
ideation,190.0,161.463158,170.503289,6.0,54.0,98.5,214.00,998.0
indicator,129.0,135.333333,144.534238,1.0,44.0,90.0,188.00,1120.0
